# ReAct Agents in LangGraph: Weather + Calendar

Objectives of this notebook:

- Build a **ReAct agent** using two tools: `weather` and `calendar`.
- Build **the same behavior** with **LangGraph** (but explicit workflow this time).
- Compare both approaches on several user queries.
- Observability with Langfuse (local)

## 0. Graph Setup

We use:
- a local LLM via **Ollama** or **LM Studio** (configured in `.env`)
- two **simulated** tools (no external API):
  - `fake_get_weather`: simple weather
  - `fake_calendar_query`: simulated agenda slots

In [ ]:
from typing import TypedDict, Optional, List, Literal
from datetime import datetime

import os
from dotenv import load_dotenv

from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_react_agent, AgentExecutor

from langgraph.graph import StateGraph, END

# Load environment variables from .env file
load_dotenv()

# Local LLM configuration
llm_type = os.getenv("LOCAL_LLM_TYPE", "ollama")

if llm_type == "ollama":
    from langchain_ollama import ChatOllama
    
    ollama_base_url = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")
    ollama_model = os.getenv("OLLAMA_MODEL", "llama3.2:latest")
    
    llm = ChatOllama(
        model=ollama_model,
        base_url=ollama_base_url,
        temperature=0,
    )
    print(f"Using Ollama with model {ollama_model}")
    
elif llm_type == "lmstudio":
    from langchain_openai import ChatOpenAI
    
    lmstudio_base_url = os.getenv("LMSTUDIO_BASE_URL", "http://localhost:1234/v1")
    lmstudio_model = os.getenv("LMSTUDIO_MODEL", "local-model")
    
    llm = ChatOpenAI(
        model=lmstudio_model,
        base_url=lmstudio_base_url,
        api_key="not-needed",  # LM Studio doesn't require an API key
        temperature=0,
    )
    print(f"Using LM Studio with model {lmstudio_model}")
    
else:
    raise ValueError(f"Unrecognized LOCAL_LLM_TYPE: {llm_type}. Use 'ollama' or 'lmstudio'.")

# Optional Langfuse configuration for observability
# If you have configured LANGFUSE_PUBLIC_KEY and LANGFUSE_SECRET_KEY in your .env,
# observability will be automatically enabled
if os.getenv("LANGFUSE_PUBLIC_KEY") and os.getenv("LANGFUSE_SECRET_KEY"):
    try:
        from langfuse.callback import CallbackHandler
        
        langfuse_handler = CallbackHandler(
            public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
            secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
            host=os.getenv("LANGFUSE_BASE_URL", "http://localhost:3000")
        )
        print(f"Langfuse enabled on {os.getenv('LANGFUSE_BASE_URL', 'http://localhost:3000')}")
    except ImportError:
        print("Langfuse not installed. To enable it: pip install langfuse")
        langfuse_handler = None
else:
    langfuse_handler = None
    print("Langfuse not configured (optional)")

In [ ]:

# Simulated weather tool
def fake_get_weather(period: str) -> str:
    """Simulates a weather API for 'morning', 'afternoon', 'evening' or 'day'."""
    if period == "morning" or period == "matin":
        return "This morning: 4°C, feels like 0°C, overcast sky."
    elif period == "afternoon" or period == "apres-midi":
        return "This afternoon: 16°C, rain starting at 3pm."
    elif period == "evening" or period == "soir":
        return "This evening: 9°C, thunderstorms."
    else:
        return "Today: between 4°C and 16°C, with some clear skies."

# Simulated calendar: available slots for Julie
FAKE_SLOTS = [
    {"day": "thursday", "slot": "2pm-3pm", "person": "julie"},
    {"day": "thursday", "slot": "4pm-5pm", "person": "gabriel"},
    {"day": "friday", "slot": "10am-11am", "person": "julie"},
]

def fake_calendar_query(person: str , day: Optional[str] = None) -> str:
    """Returns simulated slots for a given person."""
    slots = [s for s in FAKE_SLOTS if s["person"].lower() == person.lower()]
    if day:
        slots = [s for s in slots if s["day"].lower() == day.lower()]
    if not slots:
        return f"No slots available for {person}."
    return f"Available slots for {person}: " + ", ".join(
        f"{s['day']} {s['slot']}" for s in slots
    )


In [ ]:
QUERIES = [
    "I want to know the weather for this afternoon.",
    "Can you suggest a slot with Julie this week?",
    "If it's nice weather Thursday afternoon, suggest me a slot with Julie.",
]

## 1. ReAct Agent (practical black box)

We create a LangChain ReAct agent with two tools:

- `weather_tool(period: str)` → calls `fake_get_weather`
- `calendar_tool(person: str, day: Optional[str])` → calls `fake_calendar_query`

The framework drives the **Thought → Action → Observation** loop.


In [ ]:

@tool
def weather_tool(period: str) -> str:
    """Gives simulated weather for 'morning', 'afternoon', 'evening' or 'day'."""
    return fake_get_weather(period)

@tool
def calendar_tool(person: str = "Julie", day: Optional[str] = None) -> str:
    """Returns simulated slots for a person (default Julie)."""
    return fake_calendar_query(person=person, day=day)

tools = [weather_tool, calendar_tool]


In [ ]:
# Using LangChain's standard ReAct prompt
# This prompt is designed to work correctly with the ReAct parser
try:
    from langchain import hub
    # Retrieving the standard ReAct prompt from LangChain Hub
    prompt = hub.pull("hwchase17/react")
except Exception:
    # If hub.pull doesn't work, we use a custom prompt with the exact ReAct format
    from langchain_core.prompts import PromptTemplate
    template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""
    prompt = PromptTemplate.from_template(template)

react_agent = create_react_agent(llm, tools, prompt)

# Configure AgentExecutor with Langfuse if available
executor_config = {
    "agent": react_agent,
    "tools": tools,
    "verbose": True,
    "handle_parsing_errors": True
}

# Add Langfuse callback if configured
if langfuse_handler:
    executor_config["callbacks"] = [langfuse_handler]

react_executor = AgentExecutor(**executor_config)

In [ ]:
print("### ReAct Agent Demo (black box) ###")
for q in QUERIES:
    print(f"\n=== Question: {q}")
    # Prepare configuration with Langfuse callbacks if available
    invoke_config = {"input": q}
    if langfuse_handler:
        invoke_config["callbacks"] = [langfuse_handler]
    
    result = react_executor.invoke(invoke_config)
    print("Final answer (ReAct):", result["output"])

## 4. LangGraph Agent (explicit workflow)

We rebuild the **same behavior** with LangGraph:

1. Interpret the request (intent + parameters)
2. Decide which tools to call (weather, calendar, both)
3. Call the tools
4. Compose the final answer


In [ ]:

IntentType = Literal["weather", "calendar", "both", "none"]

class AgentState(TypedDict):
    user_input: str
    intent: IntentType
    period: Optional[str]
    person: Optional[str]
    day: Optional[str]
    weather_result: Optional[str]
    calendar_result: Optional[str]
    final_answer: Optional[str]


In [ ]:

def interpret_request(state: AgentState) -> AgentState:
    """Uses the LLM to classify the request and extract some parameters."""
    user_input = state["user_input"]
    prompt = ChatPromptTemplate.from_template(
        """You analyze the user request and return a JSON.

Request: {user_input}

Respond ONLY with a JSON of the form:
{{
  "intent": "weather" | "calendar" | "both" | "none",
  "period": "morning" | "afternoon" | "evening" | "day" | null,
  "person": string or null,
  "day": string or null
}}
"""
    )
    messages = prompt.format_messages(user_input=user_input)
    raw = llm.invoke(messages).content
    import json
    try:
        parsed = json.loads(raw)
    except json.JSONDecodeError:
        parsed = {"intent": "none", "period": None, "person": None, "day": None}

    state["intent"] = parsed.get("intent", "none")
    state["period"] = parsed.get("period")
    state["person"] = parsed.get("person")
    state["day"] = parsed.get("day")
    return state


In [ ]:

def call_weather_node(state: AgentState) -> AgentState:
    period = state.get("period") or "day"
    state["weather_result"] = fake_get_weather(period)
    return state


In [ ]:

def call_calendar_node(state: AgentState) -> AgentState:
    person = state.get("person")
    day = state.get("day")
    state["calendar_result"] = fake_calendar_query(person=person, day=day)
    return state


In [ ]:

from typing import List

def build_final_answer(state: AgentState) -> AgentState:
    parts: List[str] = []
    if state.get("weather_result"):
        parts.append(state["weather_result"])
    if state.get("calendar_result"):
        parts.append(state["calendar_result"])
    if not parts:
        parts.append("I'm not sure I understand your request.")

    state["final_answer"] = " ".join(parts)
    return state


In [ ]:

graph = StateGraph(AgentState)

graph.add_node("interpret", interpret_request)
graph.add_node("weather", call_weather_node)
graph.add_node("calendar", call_calendar_node)
graph.add_node("final", build_final_answer)

graph.set_entry_point("interpret")

def route_from_intent(state: AgentState):
    intent = state.get("intent", "none")
    if intent == "weather":
        return "weather"
    if intent == "calendar":
        return "calendar"
    if intent == "both":
        return "weather"
    return "final"

graph.add_conditional_edges(
    "interpret",
    route_from_intent,
    {
        "weather": "weather",
        "calendar": "calendar",
        "both": "weather",
        "final": "final",
    },
)

def after_weather(state: AgentState):
    if state.get("intent") == "both":
        return "calendar"
    return "final"

graph.add_conditional_edges(
    "weather",
    after_weather,
    {
        "calendar": "calendar",
        "final": "final",
    },
)

graph.add_edge("calendar", "final")
graph.add_edge("final", END)

app = graph.compile()


In [ ]:
g = app.get_graph()

from IPython.display import Image
Image(g.draw_mermaid_png())

In [ ]:

print("### LangGraph Agent Demo (explicit workflow) ###")
for q in QUERIES:
    print(f"\n=== Question: {q}")
    initial_state: AgentState = {
        "user_input": q,
        "intent": "none",
        "period": None,
        "person": None,
        "day": None,
        "weather_result": None,
        "calendar_result": None,
        "final_answer": None,
    }
    result = app.invoke(initial_state)
    print("Final answer (LangGraph):", result["final_answer"])


## 5. Quick Comparison: ReAct vs LangGraph

On this small example:

- **ReAct**:
  - We provide tools and a ReAct prompt.
  - The framework drives the Thought → Action → Observation loop.
  - The workflow remains implicit (we guess it via logs or Langfuse).

- **LangGraph**:
  - We explicitly define the steps (interpret, weather, calendar, answer).
  - We control transitions (e.g., `intent == "both"` → weather then calendar).
  - The same agent becomes testable, observable, modifiable node by node.

In production, we can then:
- Add safeguards,
- Limit certain tools to certain paths,
- Connect Langfuse to track costs, latency and errors by node.